<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Module_8_Resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this module, we will resample all of our products to a common pixel size. Since 4 of our 7 products already have a pixel size of 30 meters, we will use 30 meters as our shared pixel size.

To accomplish this, we will use the gdal.Warp method. gdal.Warp takes the following optional arguments (among others):
* xRes
* yRes
* resampleAlg

Since we want to create square pixels measuring 30 meters by 30 meters in size, we will set xRes and yRes to both equal 30. The resampleAlg argument determines how the input map will go about determining the pixel value for each output 30x30m pixel. There are several options for this argument. In this case, we will use 'mode', which is the Mode resampling algorithm. For more information on the different resampling algorithms available in GDAL, see [this page](https://gdal.org/en/stable/programs/gdalwarp.html) and scroll down to "-r resampling_method".

# Step 1: Import packages

In [ ]:
import ee
import geemap
from google.colab import drive
import os
import glob
from osgeo import gdal
import numpy as np
import pandas as pd
import time

In [ ]:
ee.Authenticate()

ee.Initialize(project='servir-sco-assets')

# MODIFIABLE VARIABLE ALERT

In [ ]:
my_gee_folder = "users/mickymags/sep_arkansas/"
my_Gdrive_folder = "/content/drive/MyDrive/Flood_Intercomparison/Case_Studies/sep/sep_arkansas/"
other_Gdrive_folder = "/content/drive/MyDrive/Flood_Intercomparison/Case_Studies/Flood_Intercomparison/"
flood_event_desc = 'sep_arkansas'
time_id = '09092043'                # put in a string correlating to the current time which you are running the code -- will help with identifying exports

In [ ]:
aoi = ee.FeatureCollection(my_gee_folder + "aoi")
roi = aoi.geometry()
aoi_centroid = aoi.geometry().centroid()             # Get the center of the AOI
lon = aoi_centroid.coordinates().get(0).getInfo()    # Extract the longitude from the centroid
lat = aoi_centroid.coordinates().get(1).getInfo()    # Extract the latitude from the centroid

# Step 2: Resample GFM, MCDWD, and VFM products (and maybe HYDROSAR)

In order to run object extraction statistics, we must have all products in a common projection. Thus, we will use gdal.Warp to resample these products to 30 meters

In [ ]:
drive.mount('/content/drive/')

In [ ]:
os.chdir(my_Gdrive_folder)

In [ ]:
pwd

In [ ]:
ls

## Step 2 Part 1: GFM

In [ ]:
os.chdir('GFM')

In [ ]:
ls m*

In [ ]:
infile = glob.glob('merged*')[0]

In [ ]:
infile

In [ ]:
outfile = 'gfm_resampled_' + flood_event_desc + '.tif'

In [ ]:
info = gdal.Info(infile)
find = info.find('Data axis')
proj = info[find-8:find-3]
my_proj = 'EPSG:'+proj
print(my_proj)

In [ ]:
gdal.Warp(outfile, infile, dstSRS=my_proj, resampleAlg='near', xRes=30, yRes=30)

In [ ]:
ls gfm_re*

## Step 2 Part 2: MCDWD

In [ ]:
os.chdir('../MCDWD')

In [ ]:
pwd

In [ ]:
mcdwd_infile = infile = glob.glob('merged*')[0]

In [ ]:
mcdwd_infile

In [ ]:
ls m*

In [ ]:
mcdwd_outfile = 'mcdwd_resampled_' + flood_event_desc + '.tif'

In [ ]:
gdal.Warp(mcdwd_outfile, mcdwd_infile, dstSRS=my_proj, resampleAlg='near', xRes=30, yRes=30)

In [ ]:
ls mcdwd_re*

## Step 2 Part 3: VFM

In [ ]:
os.chdir('../VFM')

In [ ]:
pwd

In [ ]:
vfm_infile = infile = glob.glob('merged*')[0]

In [ ]:
vfm_outfile = 'vfm_resampled_' + flood_event_desc + '.tif'

In [ ]:
vfm_infile

In [ ]:
gdal.Warp(vfm_outfile, vfm_infile, dstSRS=my_proj, resampleAlg='near', xRes=30, yRes=30)

In [ ]:
ls v*

In [ ]:
pwd

## Step 2 Part 4: HYDROSAR

In some cases, HYDROSAR will not have a pixel size of 30 meters. If it does not, we will undergo the same workflow we did for VFM, GFM, and MCDWD. Let's check the pixel size of HYDROSAR

In [ ]:
os.chdir('../HYDROSAR')

In [ ]:
hydrosar_infile = infile = glob.glob('merged*')[0]

In [ ]:
hydrosar_file = gdal.Open(hydrosar_infile)

hydrosar_pixel_size = hydrosar_file.GetGeoTransform()[1]

if hydrosar_pixel_size == 30:
  print("HYDROSAR has a pixel size of 30 meters. Skip to step 3.")
else:
  print("HYDROSAR has a pixel size slightly different than 30 meters. Continue running the code below.")

In [ ]:
hydrosar_outfile = 'hydrosar_resampled_' + flood_event_desc + '.tif'

In [ ]:
gdal.Warp(hydrosar_outfile, hydrosar_infile, dstSRS=my_proj, resampleAlg='near', xRes=30, yRes=30)

# Step 3: Run Harmonization Module again

For GFM, MCDWD, and VFM, (and maybe HYDROSAR), download the images to your Google Drive. Then, upload the resampled images to Google Earth Engine. For each image, make the asset id equivalent to the google earth engine folder, followed by "x_resampled_mosaic" where x is the name of your flood product. Now we will rerun the harmonization module on each image

In [ ]:
mcdwd_resampled = ee.Image(my_gee_folder + 'mcdwd_resampled_mosaic')
gfm_resampled = ee.Image(my_gee_folder + 'gfm_resampled_mosaic')
vfm_resampled = ee.Image(my_gee_folder + 'vfm_resampled_mosaic')

## Step 3 Part 1: VFM Resampled Harmonization

In [ ]:
os.chdir('../VFM')

input_vfm = glob.glob('merged_vfm*')[0]
inter_vfm = f'harmonized_vfm_{flood_event_desc}.tif'

in_vfm = gdal.Open(input_vfm)
vfm_band = in_vfm.GetRasterBand(1)
vfm_data = vfm_band.ReadAsArray()

################# Set Nonwater Pixels to 0  ################
# Pixel Values of 16 indicate clear-sky bare land
vfm_data[vfm_data == 16] = 0

# Pixel Values of 17 indicate clear-sky vegetation
vfm_data[vfm_data == 17] = 0

# Pixel Values of 20 indicate snow cover
vfm_data[vfm_data == 20] = 0

# Pixel Values of 25 indicate ice
vfm_data[vfm_data == 25] = 0

# Pixel Values of 27 indicate river/lake covered in ice
vfm_data[vfm_data == 27] = 0

# Pixel Values of 38 indicate supra-snow ice water, mixed ice and water, or melting ice
vfm_data[vfm_data == 38] = 0

# Pixel Values between 100 and 150 indicate the relative percentage of a flooded pixel
# that has been identified to be water. 101 corresponds to 1% flooded, 150 corresponds to 50% flooded, etc.
vfm_data[(vfm_data > 100) & (vfm_data < 150)] = 0

###################### Set Masked Pixels to 2 ####################
# Pixel Values of 1 indicate bad data pixels
vfm_data[vfm_data == 1] = 2

# Pixel Values of 30 indicate pixels where there is cloud cover
vfm_data[vfm_data == 30] = 2

# Pixel Values of 50 indicte pixels that have shadows from clouds or terrain
vfm_data[vfm_data == 50] = 2


############### Set Water Pixels to 1 #########################
# Pixel Values of 15 indicate open water without water fraction retrieval
vfm_data[vfm_data == 15] = 1

# Pixel Values of 99 indicate open normal water from a river, lake, reservation, ocean
vfm_data[vfm_data == 99] = 1

# Pixel values greater than 150 indicate a pixel is 50% flooded or more
vfm_data[vfm_data >= 150] = 1

##### Create Output Raster Using The Same Driver and Metadata ##########
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(
    inter_vfm,
    in_vfm.RasterXSize,
    in_vfm.RasterYSize,
    1,
    vfm_band.DataType
)

# Copy Spatial Metadata
out_ds.SetGeoTransform(in_vfm.GetGeoTransform())
out_ds.SetProjection(in_vfm.GetProjection())

# Write Modified Array
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(vfm_data)

# Clean up
out_band.FlushCache()
in_ds = None
out_ds = None

## Step 3 Part 2: GFM Resampled Harmonization

In [ ]:
os.chdir('../GFM')

inter_gfm = f'merged_gfm_{flood_event_desc}.tif'
final_gfm = f'harmonized_gfm_{flood_event_desc}.tif'
vector_gfm = glob.glob('drive_export*')[0]

gdal.Warp(
    final_gfm,
    inter_gfm,
    cutlineDSName=vector_gfm,
    cropToCutline=True,
    dstNodata = 2
)

## Step 3 Part 3: MCDWD Resampled Harmonization

In [ ]:
os.chdir('../MCDWD')

input_mcdwd = glob.glob('merged_mcdwd*')[0]
inter_mcdwd = f'harmonized_mcdwd_{flood_event_desc}.tif'

in_mcdwd = gdal.Open(input_mcdwd)
mcdwd_band = in_mcdwd.GetRasterBand(1)
mcdwd_data = mcdwd_band.ReadAsArray()

################# Set Nonwater Pixels to 0  ################
# Nonwater is already set to 0 by the MCDWD algorithm

############### Set Water Pixels to 1 #########################
# Pixel Values of 1 indicate regular surface water according to the MCDWD algorithm
# We do not need to alter these values

# Pixel Values of 2 indicate a recurring flood according to the MCDWD algorithm
mcdwd_data[mcdwd_data == 2] = 1

# Pixel Values of 3 indicate floodwater according to the MCDWD algorithm
mcdwd_data[mcdwd_data == 3] = 1

###################### Set Masked Pixels to 2 ####################
## More research needed on masked pixels, not sure how to access mask value in GDAL. .GetMaskBand() only returns an array of all 255

##### Create Output Raster Using The Same Driver and Metadata ##########
driver = gdal.GetDriverByName('GTiff')     # Get the GTiff Driver
out_ds = driver.Create(                    # Use the Create method to create the dataset
    inter_mcdwd,
    in_mcdwd.RasterXSize,                 # Get the X Size of the Input DSWx-HLS Image
    in_mcdwd.RasterYSize,                 # Get the Y Size of the Input DSWx-HLS Image
    1,                                     # Get the number of bands of the Input DSWx-HLS Image
    mcdwd_band.DataType                   # Get the Datatype of the input DSWx-HLS Image
)

# Copy Spatial Metadata
out_ds.SetGeoTransform(in_mcdwd.GetGeoTransform())   # Set the geotransform of the output to be the same as the input image
out_ds.SetProjection(in_mcdwd.GetProjection())       # Set the projection of the output to be the same as the input image

# Write Modified Array
out_band = out_ds.GetRasterBand(1)                    # Get the first band of the output dataset
out_band.WriteArray(mcdwd_data)                      # Write the modified array to this band of the output dataset.

# Clean up
out_band.FlushCache()
in_ds = None
out_ds = None

## Step 3 Part 4: HYDROSAR

Only run the code cell below if you had to resample HYDROSAR in Step 2 Part 4.

## Step 4: Exporting to Google Earth Engine

In [ ]:
proj = mcdwd_resampled.projection().getInfo()['crs']

# Define a function that exports an Image to a Google Earth Engine Asset.
def exporter(img, asset_id):

  desc = 'Flood_Map_Export_'
  region_ = aoi.geometry()
  geemap.ee_export_image_to_asset(image = img,
                                  assetId = asset_id,
                                  description = desc,
                                  region = region_,
                                  crs = proj,
                                  scale = 30,
                                  maxPixels = 1e13)
  return 0

In [ ]:
# Export VFM
vfm_resampled_harmonized_aid = my_gee_folder + 'vfm_resampled_harmonized'
exporter(vfm_resampled_final, vfm_resampled_harmonized_aid)

# Export GFM
gfm_resampled_harmonized_aid = my_gee_folder + 'gfm_resampled_harmonized'
exporter(gfm_resampled_final, gfm_resampled_harmonized_aid)

# Export MCDWD
mcdwd_resampled_harmonized_aid = my_gee_folder + 'mcdwd_resampled_harmonized'
exporter(mcdwd_resampled_final, mcdwd_resampled_harmonized_aid)

If you had to resample HYDROSAR